# SkinAndBlood

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'skinandblood'
data_type = 'methylation'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2018
approved_by_author = '⌛'
citation = "Horvath, Steve, et al. \"Epigenetic clock for skin and blood cells applied to Hutchinson Gilford Progeria Syndrome and ex vivo studies.\" Aging (Albany NY) 10.7 (2018): 1758."
doi = "https://doi.org/10.18632/aging.101508"
notes = None

## Download necessary data

#### Download directly with curl

In [3]:
supplementary_url = "https://www.aging-us.com/article/101508/supplementary/SD5/0/aging-v10i7-101508-supplementary-material-SD5.csv"
supplementary_file_name = "coefficients.csv"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

## Load data

#### From CSV file

In [4]:
df = pd.read_csv('coefficients.csv')

## Extract features and weights

First, let's extract the features and weights:

In [5]:
df['feature'] = df['ID']
df['coefficient'] = df['Coef']

df.head()

,ID,Coef,corAgeTraining,chr,pos,strand,UCSC_RefGene_Name,UCSC_RefGene_Accession,UCSC_RefGene_Group,Phantom4_Enhancers,...,X450k_Enhancer,HMM_Island,Regulatory_Feature_Name,OpenChromatin_NAME,TFBS_NAME,Methyl27_Loci,Methyl450_Loci,CpGnumber,feature,coefficient
0,(Intercept),-0.447119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(Intercept),-0.447119
1,cg12140144,0.363181,0.371780,chr1,2984275.0,-,FLJ42875;PRDM16;FLJ42875;PRDM16,NR_024371;NM_199454;NR_015440;NM_022114,Body;TSS1500;Body;TSS1500,NaN,...,NaN,1:2973786-2977805,NaN,chr1:2976517-2991800,NaN,NaN,True,1.0,cg12140144,0.363181
2,cg26933021,-0.090500,-0.569687,chr1,12618574.0,-,NaN,NaN,NaN,NaN,...,True,1:12541103-12541183,NaN,NaN,NaN,NaN,True,2.0,cg26933021,-0.090500
3,cg20822990,-0.007025,0.519172,chr1,17338766.0,-,ATP13A2;ATP13A2;ATP13A2,NM_022089;NM_001141974;NM_001141973,TSS1500;TSS1500;TSS1500,NaN,...,True,NaN,1:17336920-17338827,NaN,NaN,NaN,True,3.0,cg20822990,-0.007025
4,cg07312601,-0.135092,-0.422485,chr1,19583887.0,-,MRTO4,NM_016183,Body,NaN,...,NaN,NaN,1:19583679-19583890,NaN,NaN,NaN,True,4.0,cg07312601,-0.135092


Then, let's create lists for features and weights. Be careful about the intercept, as it usually shows up as a feature name.

In [6]:
features = df['feature'][1:].tolist()
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

## Load weights into pyaging model

#### Linear model

In [7]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights.float()
model.linear.bias.data = intercept.float()

model

LinearModel(
  (linear): Linear(in_features=391, out_features=1, bias=True)
)

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [8]:
reference_feature_values = None

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [9]:
preprocessing = None

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [10]:
def postprocessing_function(x):
    """
    Applies an anti-logarithmic linear transformation to a PyTorch tensor.
    """
    adult_age=20
    
    if x < 0:
        # Exponential transformation for negative values
        age_tensor = (1 + adult_age) * math.exp(x) - 1
    else:
        # Linear transformation for non-negative values
        age_tensor = (1 + adult_age) * x + adult_age
    return age_tensor
    
postprocessing_function_string = marshal.dumps(postprocessing_function.__code__)

postprocessing_helper_objects = None

postprocessing = {
    'name': 'anti_log_linear',
    'postprocessing_function': postprocessing_function_string,
    'postprocessing_helper_objects': postprocessing_helper_objects
}

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [11]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [12]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [13]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [14]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [15]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [16]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [17]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: coefficients.csv
Deleted folder: .ipynb_checkpoints
